In [1]:
import torch
import pytorch_lightning as pl
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
import torchaudio
from src.dataloader import MultiFXDataset
from src.model.resnet import resnet18
import pytorch_lightning as pl
from argparse import ArgumentParser
from logging import Logger
from warnings import simplefilter

In [2]:
data = "dataset/generated/gen_multiFX_10102021"
with_clean = True
batch_size = 128
learning_rate = 1e-3

trainer_args = {
    "gpus": 1,
    "max_epochs": 10,
}

In [ ]:
transform = torchaudio.transforms.MelSpectrogram(sample_rate=44100,
                                                  n_fft=2048,
                                                  n_mels=128).to("cuda")
train_set = MultiFXDataset(data, 'train', None)
valid_set = MultiFXDataset(data, 'valid', None)
train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=16, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, num_workers=16)

print("=> Start training")
with_clean = True
if with_clean is True:
    print("=> Training with clean")
    in_channels = 2
else:
    print("=> Training no clean")
    in_channels = 1

model = resnet18(in_channels, train_set.settings["n_classes"], with_clean, learning_rate, transform=transform)

trainer = pl.Trainer(**trainer_args)
trainer.fit(model, train_loader, valid_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


=> Start training
=> Training with clean


2021-10-10 18:35:53.467278: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

   | Name      | Type              | Params
-------------------------------------------------
0  | transform | MelSpectrogram    | 0     
1  | conv1     | Conv2d            | 800   
2  | bn1       | BatchNorm2d       | 32    
3  | relu      | ReLU              | 0     
4  | maxpool   | MaxPool2d         | 0     
5  | layer_1   | Sequential        | 33.1 K
6  | layer_2   | Sequential        | 21.8 K
7  | layer_3   | Sequential        | 131 K 
8  | layer_4   | Sequential        | 525 K 
9  | avgpool   | AdaptiveAvgPool2d | 0     
10 | fc        | Linear            | 1.7 K 
-------------------------------------------------
714 K     Trainable params
0         Non-trainable params
714 K     Total params
2.859     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]